# 딥 러닝을 이용한 자연어 처리 입문

아래 링크의 E-book을 보고 실습한 내용입니다.

WikiDocs 주소: https://wikidocs.net/31766


# 6장 토픽 모델링

## 1절 잠재 의미 분석 (Latent Semantic Analysis, LSA)

## 뉴스 그룹 데이터를 활용한 LSA 실습

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(
    shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

print("뉴스 그룹 데이터의 개수:", len(documents))


뉴스 그룹 데이터의 개수: 11314


In [2]:
print("뉴스 그룹 데이터 예시 1:\n", documents[1])

뉴스 그룹 데이터 예시 1:
 






Yeah, do you expect people to read the FAQ, etc. and actually accept hard
atheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out
of steam!







Jim,

Sorry I can't pity you, Jim.  And I'm sorry that you have these feelings of
denial about the faith you need to get by.  Oh well, just pretend that it will
all end happily ever after anyway.  Maybe if you start a new newsgroup,
alt.atheist.hard, you won't be bummin' so much?






Bye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) 
--
Bake Timmons, III


In [3]:
print("뉴스 그룹 데이터가 가지고 있는 카테고리:\n", dataset.target_names)

뉴스 그룹 데이터가 가지고 있는 카테고리:
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### 텍스트 전처리

In [4]:
news_df = pd.DataFrame({"document": documents})

# 특수 문자 제거 - 영문자가 아니면 모두 공백으로 치환한다.
news_df['clean_doc'] = news_df['document'].str.replace(
    r"[^a-zA-Z]", " ", regex=True)

# 길이가 짧은 단어 제거 - 길이가 3이하면 제거
# 단어들 모두 소문자로 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(
    lambda x: ' '.join([w.lower() for w in x.split() if len(w) > 3]))

# 데이터 정제 후 결과 확인
news_df['clean_doc'][1]


'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [5]:
# 토큰화 수행 후 불용어 제거
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(
    lambda x: [item for item in x if item not in stop_words])

# 불용어 제거 후 토큰화 되어 있는 결과 확인
print(tokenized_doc[1])


['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


In [6]:
# TF-IDF 행렬을 만들기 위해 역토큰화 진행
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
news_df['clean_doc'] = detokenized_doc

# 역토큰화 결과 확인
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

In [7]:
# TF-IDF 행렬 만들기
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words='english', 
    max_features=1000, # 상위 1000개 단어만 보존 
    max_df = 0.5,
    smooth_idf=True,
)

X = vectorizer.fit_transform(news_df['clean_doc'])
print("X의 크기:", X.shape)

X의 크기: (11314, 1000)


### 토픽 모델링

- 토픽 모델링을 통해 해당 문서의 주제를 파악해본다.  
- 본래 20개의 카테고리를 가졌기 때문에 20개의 토픽이 존재한다고 가정하고 진행한다.


- scikit-learn의 TruncatedSVD를 사용하여 차원축소 및 토픽 모델링을 시도한다.

In [8]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd.fit(X)
print("토픽의 수:",len(svd.components_))

토픽의 수: 20


In [9]:
import numpy as np

print("SVD의 V^T에 해당하는 svd.components_,\nshape:", np.shape(svd.components_), "= (토픽의 수, 단어의 수)")

SVD의 V^T에 해당하는 svd.components_,
shape: (20, 1000) = (토픽의 수, 단어의 수)


In [10]:
# 단어들의 집합, 1000개의 단어 (위에서 상위 1000개만 보존하도록 설정함)
terms = vectorizer.get_feature_names()


def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        features = [(feature_names[i], topic[i].round(5))
                    for i in topic.argsort()[:-n - 1: -1]]
        print(f"Topic: {idx+1},\t{features}")


get_topics(svd.components_, terms)


Topic: 1,	[('like', 0.21386), ('know', 0.20046), ('people', 0.19293), ('think', 0.17805), ('good', 0.15128)]
Topic: 2,	[('thanks', 0.32888), ('windows', 0.29088), ('card', 0.18069), ('drive', 0.17455), ('mail', 0.15111)]
Topic: 3,	[('game', 0.37064), ('team', 0.32443), ('year', 0.28154), ('games', 0.2537), ('season', 0.18419)]
Topic: 4,	[('drive', 0.53324), ('scsi', 0.20165), ('hard', 0.15628), ('disk', 0.15578), ('card', 0.13994)]
Topic: 5,	[('windows', 0.40399), ('file', 0.25436), ('window', 0.18044), ('files', 0.16078), ('program', 0.13894)]
Topic: 6,	[('chip', 0.16114), ('government', 0.16009), ('mail', 0.15625), ('space', 0.1507), ('information', 0.13562)]
Topic: 7,	[('like', 0.67086), ('bike', 0.14236), ('chip', 0.11169), ('know', 0.11139), ('sounds', 0.10371)]
Topic: 8,	[('card', 0.46633), ('video', 0.22137), ('sale', 0.21266), ('monitor', 0.15463), ('offer', 0.14643)]
Topic: 9,	[('know', 0.46047), ('card', 0.33605), ('chip', 0.17558), ('government', 0.1522), ('video', 0.14356)]

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
